<a href="https://colab.research.google.com/github/gtbnhyujmj/BingX_Auto_Order/blob/main/BingX_Bot_Me_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install requests

In [ ]:
import time                # 引入時間模組，用來取得現在時間
import requests              # 引入requests模組，讓我們可以發送網路請求
import hmac                # 引入hmac模組，幫忙做簽名加密
from hashlib import sha256        # 引入sha256加密演算法

# V2

In [ ]:
def demo():
    # 這個函式示範怎麼叫API查帳戶餘額
    path = 'https://open-api.bingx.com/openApi/swap/v2/user/balance'      # API的路徑 V2
    method = "GET"                               # 用GET方法請求資料

    # A = params["recvWindow"]
    # B = params["timestamp"]

    # params_str = "recvWindow=" + A + "&" + "timestamp=" + B   # 把字典格式的參數變成字串

    return method, path, # params_str  # 呼叫發送請求的函式，並回傳結果

method, path = demo()
demo()

('GET', 'https://open-api.bingx.com/openApi/swap/v2/user/balance')

# Params

In [ ]:
params = {
        "recvWindow": "60000",                    # 允許的請求時間誤差，60秒（毫秒）
        "timestamp": str(int(time.time() * 1000))          # 現在時間戳記（毫秒），用來驗證請求有效
    }

# 排序後連結成URL的後半部

In [ ]:
def format_params(params):
    # >>> 這個函式把參數排好序，然後組成a=1&b=2這樣的字串

    keys = sorted(params.keys())  # 把參數的鍵排序

    # 建立一個空清單準備放字串
    param_list = []
    for key in keys:
        # 這裡用簡單字串串接，把key跟value串成 "key=value" 的形式
        param_list.append(key + "=" + params[key])

    # 用 & 把所有參數連起來
    return "&".join(param_list)

final_paramas = format_params(params)
format_params(params)  ## 這個就是網頁上的 params_str = 把參數排好序，然後組成a=1&b=2這樣的字串

'recvWindow=60000&timestamp=1747285369087'

# 密鑰

In [ ]:
SECRETKEY = ""

# 創造簽證

In [ ]:
def get_signature(secret, message): # message = 上面處理好的字串
    # 這裡用HMAC-SHA256演算法，根據密鑰跟參數字串做簽名

    sign = hmac.new(secret.encode(), message.encode(), sha256).hexdigest()
    # print("簽名:", sign)      # 印出簽名結果方便檢查
    return sign           # 回傳簽名字串

sign = get_signature(SECRETKEY, final_paramas)
get_signature(SECRETKEY, final_paramas)
# print("簽名:", get_signature)

'707f6ebbe16c2478c3c6bd79644853351032f96b732f9180c673d5c0c7df4fa2'

# 串成完整的URL

In [ ]:
# 把完整網址組合起來（字串串接方式）
url = path + "?" + final_paramas + "&signature=" + sign

# url

'https://open-api.bingx.com/openApi/swap/v2/user/balance?recvWindow=60000&timestamp=1747285369087&signature=707f6ebbe16c2478c3c6bd79644853351032f96b732f9180c673d5c0c7df4fa2'

# 用公鑰組成Headers

In [ ]:
# 設定HTTP請求的標頭，把APIKEY放進去

APIKEY = ""

In [ ]:
headers = {
        'X-BX-APIKEY': APIKEY
    }

# 接收資料

In [ ]:
# 用requests送出HTTP請求
# method = GET, 函數上面有

response = requests.request(method, url, headers=headers)
response.status_code

200

# 列印JSON

In [ ]:
response = requests.request(method, url, headers=headers)
response.text

'{"code":0,"msg":"","data":{"balance":{"userId":"1049889271413661698","asset":"USDT","balance":"0.1568","equity":"0.1568","unrealizedProfit":"0.0000","realisedProfit":"0","availableMargin":"0.1568","usedMargin":"0.0000","freezedMargin":"0.0000","shortUid":"8883392"}}}'

In [ ]:
# 自動排版

import json  # 加在檔案開頭

# 在 print(result) 改成這樣：
parsed = json.loads(response.text)
print(json.dumps(parsed, indent=2))

{
  "code": 0,
  "msg": "",
  "data": {
    "balance": {
      "userId": "1049889271413661698",
      "asset": "USDT",
      "balance": "0.1568",
      "equity": "0.1568",
      "unrealizedProfit": "0.0000",
      "realisedProfit": "0",
      "availableMargin": "0.1568",
      "usedMargin": "0.0000",
      "freezedMargin": "0.0000",
      "shortUid": "8883392"
    }
  }
}


# 紀錄

In [ ]:
# "asset": "USDT",
# "balance": "0.1568",

# 這個跟永續U合約的資料是符合的 USDT = 0.15684562
# 理財帳戶跟標準合約 BX那邊不開放 所以沒資料也合理

# 我把USDC轉進去看看數字會不會改變
# 沒有 可是USDC是永續U本位捏 怪怪的

# 把USDT轉出0.1U看看
# 2025/05/15 成功 >>> 資料變成0.0568U  >>> APP show 0.05684562

# V2 API OK

In [ ]:
# STU